XGboost на dense без групповыхз фичей, без таймстемпов, 500 деревьев 
    
kaggle 0.01559 0.01556 (baseline 0.01554)

cv  0.014577580587445576 +- 5.6226181232644126e-05


XGBoost на sparse, 500 деревьев переходит бейзлайн

kaggle 0.01550 0.01547

In [2]:
import numpy as np
from xgboost import XGBClassifier as xgb
from tqdm import tqdm
import os
import xgboost
from sklearn.metrics import log_loss

In [13]:
def make_sub(pred, name_suffix='', sub_dir = 'submitions/'):
    n = len(os.listdir(sub_dir))
    f_name = sub_dir  + 'sub' + str(n) + '_' + name_suffix
    print(f_name)
    
    f = open(f_name, 'w')
    f.write('Id,Click' + '\n')
    for i, prob in tqdm(enumerate(pred)):
        f.write(str(i + 1) + "," + str(prob) + '\n')
    f.close()

In [45]:
y = np.load(open('train.y', 'rb'))[:, 0]

In [3]:
#Классы несбалансированны
num1 = np.sum(y == 1)
num0 = np.sum(y == 0)
num0, num1

(29913451, 76302)

In [4]:
#nagative sampling
sample_nums_1 = np.random.choice(np.where(y == 1)[0], size = num1)
sample_nums_0 = np.random.choice(np.where(y == 0)[0], size = num1)
sample_nums = np.concatenate([sample_nums_1, sample_nums_0])

In [44]:
%%time
X = np.load(open('train.x', 'rb'))

CPU times: user 8.39 s, sys: 10.6 s, total: 19 s
Wall time: 29 s


In [6]:
X = X[sample_nums]

In [7]:
X.dump(open('train_neg_sample.x', 'wb'))

In [27]:
y = y[sample_nums]
y.dump(open('train_neg_sample.y', 'wb'))

In [15]:
model = xgb()

In [21]:
X.shape

(152604, 14)

In [22]:
keys = "timestamp;label;C1;C2;C3;C4;C5;C6;C7;C8;C9;C10;CG1;CG2;CG3;l1;l2;C11;C12".split(";")

In [29]:
model.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [30]:
p = model.predict_proba(X)

In [49]:
#make crossval set
cv_size = 0.05
cvdir = 'crossval_set/'
n_folds = 5

for i in tqdm(range(n_folds)):
    sample_nums_1 = np.random.choice(np.where(y == 1)[0], size = int(num1 * cv_size))
    sample_nums_0 = np.random.choice(np.where(y == 0)[0], size = int(num0 * cv_size))
    sample_nums = np.concatenate([sample_nums_1, sample_nums_0])

    X[sample_nums].dump(open(cvdir + str(i) + '.x', 'wb'))
    y[sample_nums].dump(open( cvdir + str(i) + '.y', 'wb'))

100%|██████████| 5/5 [00:11<00:00,  2.36s/it]


In [118]:
X =np.load('./train_neg_sample.x')
y =np.load('./train_neg_sample.y')

In [61]:
pred.ndim

2

In [14]:
def prob_shift(pred,num0, num1):
    r_factor = num1 / num0
    if pred.ndim == 2:
        prob_sampled = pred[:, 1]
    else:
        prob_sampled = pred
    prob = prob_sampled * r_factor / (1 + (r_factor - 1) * prob_sampled)
    return prob

In [71]:
def cross_val(model, cvdir ='crossval_set/'):
    n_folds = 5
    
    losslist = []
    for i in tqdm(range(n_folds)):
        X = np.load(open(cvdir + str(i) + '.x', 'rb'))
        y = np.load(open(cvdir + str(i) + '.y', 'rb'))
        pred = model.predict_proba(X)
        losslist.append(log_loss(y, prob_shift(pred, num0, num1)))
    losslist = np.array(losslist)
    return  np.mean(losslist), np.std(losslist)

In [73]:
cross_val(model)

100%|██████████| 5/5 [00:12<00:00,  2.47s/it]


(0.015187734941411599, 6.0126294650011109e-05)

In [74]:
X_test = np.load(open('test.x', 'rb'))

In [80]:
%%time
pred = model.predict_proba(X_test)

CPU times: user 2min 28s, sys: 10 s, total: 2min 39s
Wall time: 19.8 s


In [84]:
pred = prob_shift(pred, num0, num1)

In [89]:
make_sub(pred)

kaggle 0.01599 0.01595

cv     0.015187734941411599 +- 6.0126294650011109e-05 

In [107]:
model = xgb(max_depth=5, n_estimators=500)

In [108]:
%%time
model.fit(X, y)

CPU times: user 29min 3s, sys: 5.06 s, total: 29min 8s
Wall time: 1min 59s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [132]:
cross_val(model)

100%|██████████| 5/5 [00:36<00:00,  7.40s/it]


(0.014577580587445576, 5.6226181232644126e-05)

In [137]:
%%time
pred = model.predict_proba(X_test)

CPU times: user 15min 7s, sys: 12 s, total: 15min 19s
Wall time: 59.3 s


In [138]:
pred = prob_shift(pred, num0, num1)

In [139]:
make_sub(pred, 'xgboost_n500_d5')

20317220it [00:56, 358522.02it/s]


kaggle 0.01559 0.01556

cv  0.014577580587445576 +- 5.6226181232644126e-05



случайно удалил max_depth=6 n_estimators=1000 -- переучились

kaggle 0.01569 0.01566

cv 0.014348310448679059 +- 5.2657651726660568e-05

In [148]:
model = xgb(max_depth=6, n_estimators=1000, learning_rate=0.01)

In [149]:
%%time
model.fit(X, y)

CPU times: user 1h 18min 43s, sys: 10.7 s, total: 1h 18min 54s
Wall time: 4min 38s


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [150]:
cross_val(model)

100%|██████████| 5/5 [00:56<00:00, 11.35s/it]


(0.014800733577611202, 5.8791752900808735e-05)

In [151]:
%%time
pred = model.predict_proba(X_test)

CPU times: user 34min 26s, sys: 14.4 s, total: 34min 40s
Wall time: 2min 22s


In [152]:
make_sub(prob_shift(pred, num0, num1), 'xgboost_n1000_d6_lr0.01')

38238it [00:00, 382378.54it/s]

submitions/sub4_xgboost_n1000_d6_lr0.01


20317220it [00:53, 378303.81it/s]


Сделаем sparse

In [255]:
categorical = {0,1,2,3,4,5,6,7,8,9,12, 13}
continual = {10, 11}
feature_dict = dict()

In [256]:
c = sum([len(feature_dict[x]) for x in feature_dict if x in categorical])
for i in tqdm(range(X.shape[1])):
    if i in categorical:
        if i not in feature_dict:
            feature_dict[i] = {}
        for x in np.unique(X[:, i]):
            if x not in feature_dict[i]:
                    feature_dict[i][int(x)] = str(c) + ':1 '
                    c += 1
    if i in continual:
        if i not in feature_dict: 
            feature_dict[i] = c
            c += 1

100%|██████████| 14/14 [00:00<00:00, 48.82it/s]


In [258]:
feature_dict[10]

138660

In [266]:
def make_sparse_svm(X, y, outfile = 'neg_sample.svm'):
    fsvm = open(outfile, 'w')
    for i in tqdm(range(y.shape[0])):
        line = str(y[i]) + ' '
        for j in range(X.shape[1]):
            x_val = X[i,j]
            if j in categorical:
                if x_val in feature_dict[j]:
                    line += feature_dict[j][x_val]
            if j in continual:
                line += str(feature_dict[j]) + ':' + str(x_val) + ' '
        fsvm.write(line + '\n')
    fsvm.close()

In [270]:
make_sparse_svm(X_test, np.zeros(X_test.shape[0]), 'test.svm')

100%|██████████| 20317220/20317220 [07:09<00:00, 47331.44it/s]


In [3]:
train = xgboost.DMatrix('neg_sample.svm')

In [25]:
%%time
param = {'max_depth':5, 'objective':'binary:logistic' }
bst = xgboost.train(param, train, 700)

CPU times: user 35min 59s, sys: 6.53 s, total: 36min 6s
Wall time: 3min


In [26]:
%%time
test = xgboost.DMatrix('test.svm')

CPU times: user 13.8 s, sys: 6.66 s, total: 20.5 s
Wall time: 28.6 s


In [27]:
%%time
pred = bst.predict(test)

CPU times: user 13min 24s, sys: 3.33 s, total: 13min 28s
Wall time: 47.6 s


In [28]:
pred = prob_shift(pred, num0, num1)

In [29]:
make_sub(pred, 'xgboost_sparse_n500_d5')

0it [00:00, ?it/s]

submitions/sub7_xgboost_sparse_n500_d5


20317220it [00:50, 404049.32it/s]


In [32]:
!kg submit submitions/sub7_xgboost_sparse_n500_d5 -u denbibik -p  -c online-advertising-challenge-fall-2017

0.01547
